In [2]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd

order = pd.read_csv("zamowienia.csv", sep=";", dtype={'order_id': int, 'client_id': int})
order = order.drop_duplicates()
order.head()

,order_id,client_id,order_date,KUPON
0,1462414,523933015,29.06.2019,NaN
1,1538139,814107372,15.08.2019,NaN
2,1838871,713139458,19.08.2019,NaN
3,1896544,123641536,05.05.2019,15%
4,1943852,602252913,30.07.2019,NaN


In [3]:
cost = pd.read_csv("cena bazowa.csv", sep=";")
cost[cost.duplicated()] #No duplicates
cost.head()

,item_id,base_price
0,7447126,225
1,2696692,112
2,2883136,192
3,1984910,136
4,2242676,107


In [4]:
sent = pd.read_csv("wyslane.csv", sep=";", dtype={'order_id': int})
sent[sent.duplicated()] #No duplicates
sent.head()

,order_id,item_id_1,item_id_2,item_id_3
0,314631735,2655227,9051607.0,NaN
1,802224752,1984910,4286301.0,NaN
2,716057036,2669985,1052038.0,NaN
3,125731133,5966397,NaN,NaN
4,12070864,6788106,9643817.0,NaN


In [5]:
len(order), len(cost), len(sent)

(5018, 110, 3667)

In [6]:
#1.W jakim dniu roku klienci złożyli najwięcej zamówień?
order.order_date.value_counts()

14.08.2019    41
15.04.2019    39
18.06.2019    39
06.05.2019    39
23.08.2019    38
              ..
21.05.2019    18
21.06.2019    17
08.07.2019    16
09.03.2019    14
24.09.2019     1
Name: order_date, Length: 182, dtype: int64

In [7]:
#Ad 1. Najwięcej zamówień zostało złożonych 14.08.2019r. - środa 14 sierpień 2019r. (41 zamówień)

#2)Ilu klientów skorzystało z kuponu rabatowego w trakcie zakupów?

len(order[order.KUPON.notna()]['client_id'].unique())

998

In [8]:
#Ad 2. 998 klientów skorzystało z kuponów rabatowych.

#3) Ilu klientów zrobiło w analizowanym okresie więcej niż jedno zamówienie?
order.groupby('client_id').size().value_counts() #18

1    4982
2      18
dtype: int64

In [9]:
order.groupby('client_id').size().reset_index(name='count').sort_values("count")

#Ad 3. 18 klientów złożyło więcej niż 1 zamówienie 

# 4) Który z produktów cieszył się największym powodzeniem? Ilu klientów kupiło go ze zniżką?

,client_id,count
0,107893,1
3330,661582568,1
3329,661450457,1
3328,660492612,1
3327,660429748,1
...,...,...
3641,723751188,2
3640,723604593,2
3639,723602661,2
12,2525492,2


In [10]:
id1 = sent.item_id_1.value_counts()
id2 = sent.item_id_2.value_counts()
id3 = sent.item_id_3.value_counts()

counts = [id1, id2, id3]
counts = pd.concat(counts)
counts.groupby(counts.index).sum().sort_values(ascending=False)

counts[counts.index==4355722.0]

4355722.0    39
4355722.0    30
4355722.0     1
dtype: int64

In [11]:
orders_id_with_object = sent[sent[['item_id_1','item_id_2', 'item_id_3']].isin([4355722.0]).any(axis=1)].order_id

orders_with_object = order[order.order_id.isin(orders_id_with_object)]
len(orders_with_object[orders_with_object.KUPON.notna()]['client_id'].unique())

11

In [12]:
#Ad 4. Największą popularnością cieszył się produkt o ID 4355722.0 (zamówiono go 70 razy), ze zniżką zamówiło go 11 klientów


# 5) Który produkt był najczęściej kupowany ze zniżką?
df_merged = order.merge(sent, on='order_id')
df_merged

,order_id,client_id,order_date,KUPON,item_id_1,item_id_2,item_id_3
0,1462414,523933015,29.06.2019,NaN,1115180,NaN,NaN
1,1538139,814107372,15.08.2019,NaN,7838989,3109031.0,4175443.0
2,1838871,713139458,19.08.2019,NaN,684194,NaN,NaN
3,1943852,602252913,30.07.2019,NaN,9051607,NaN,NaN
4,2143538,563294140,08.04.2019,40%,5801454,2077835.0,NaN
...,...,...,...,...,...,...,...
3662,997654864,644137182,25.07.2019,NaN,4550408,NaN,NaN
3663,998024676,305628656,20.08.2019,NaN,2469513,534353.0,NaN
3664,998261938,718872103,24.06.2019,NaN,8948103,NaN,NaN
3665,998648795,685690262,05.04.2019,NaN,6960602,NaN,NaN


In [13]:
df_merged_kupon = df_merged[df_merged.KUPON.notna()]


id1 = df_merged_kupon.item_id_1.value_counts()
id2 = df_merged_kupon.item_id_2.value_counts()
id3 = df_merged_kupon.item_id_3.value_counts()


counts = [id1, id2, id3]
counts = pd.concat(counts)
counts.groupby(counts.index).sum().sort_values(ascending=False)

#Ad 5. Produkt o ID 1554376.0 był najczęściej zamawianym produktem ze zniżką (19 razy).

#Check:
df_merged_check = order.merge(sent, on='order_id', how='left')
len(df_merged_check[(df_merged_check.KUPON.notna()) & ((df_merged_check.item_id_1==1554376.0) | (df_merged_check.item_id_2==1554376.0) | (df_merged_check.item_id_3==1554376.0))])

19

In [14]:
# 6) Jaka była końcowa wartość wszystkich zamówień w badanym okresie?

df_merged_cost = order.merge(sent, on='order_id')

for i in range(1,4):
    item_column = f'item_id_{i}'
    df_merged_cost = df_merged_cost.merge(cost, left_on=item_column, right_on='item_id', how='left')
    
df_merged_cost

,order_id,client_id,order_date,KUPON,item_id_1,item_id_2,item_id_3,item_id_x,base_price_x,item_id_y,base_price_y,item_id,base_price
0,1462414,523933015,29.06.2019,NaN,1115180,NaN,NaN,1115180,212,NaN,NaN,NaN,NaN
1,1538139,814107372,15.08.2019,NaN,7838989,3109031.0,4175443.0,7838989,74,3109031.0,172.0,4175443.0,161.0
2,1838871,713139458,19.08.2019,NaN,684194,NaN,NaN,684194,235,NaN,NaN,NaN,NaN
3,1943852,602252913,30.07.2019,NaN,9051607,NaN,NaN,9051607,208,NaN,NaN,NaN,NaN
4,2143538,563294140,08.04.2019,40%,5801454,2077835.0,NaN,5801454,138,2077835.0,189.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,997654864,644137182,25.07.2019,NaN,4550408,NaN,NaN,4550408,78,NaN,NaN,NaN,NaN
3663,998024676,305628656,20.08.2019,NaN,2469513,534353.0,NaN,2469513,139,534353.0,78.0,NaN,NaN
3664,998261938,718872103,24.06.2019,NaN,8948103,NaN,NaN,8948103,224,NaN,NaN,NaN,NaN
3665,998648795,685690262,05.04.2019,NaN,6960602,NaN,NaN,6960602,162,NaN,NaN,NaN,NaN


In [15]:
df_merged_cost['KUPON'] = df_merged_cost.KUPON.str.split('%', expand=True)
df_merged_cost['KUPON'] = df_merged_cost['KUPON'].fillna(100).astype(int)
df_merged_cost['Sum'] = df_merged_cost['base_price_x'].fillna(0)+df_merged_cost['base_price_y'].fillna(0)+df_merged_cost['base_price'].fillna(0)
df_merged_cost['Sum'] = df_merged_cost.Sum * df_merged_cost.KUPON * 0.01
df_merged_cost

df_merged_cost[['Sum']].sum()

#Ad 6. Końcowa wartość wszystkich zamówień w badanym okresie wyniosła 718875.31

Sum    718875.31
dtype: float64